In [35]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

In [36]:

# Define the URL of the website
# url = "https://www.2kratings.com"
url = "https://www.2kratings.com/"

# Make a GET request to the website
response = requests.get(
    url,
    headers={'User-Agent': 'Mozilla/5.0'}
    )

In [37]:
# Check if the response is successful
# response.status_code
soup = BeautifulSoup(response.content, "lxml")

In [38]:
team_links = soup.find_all('a', {"class": "sidebar-link py-1 mx-1"})
team_links[:3]

[<a class="sidebar-link py-1 mx-1" href="/current-teams"> <img alt="All Current Teams" class="mb-1 lazy" data-src="/wp-content/uploads/NBA-Logo-1.svg" height="20" src="/wp-content/uploads/1x1.png" title="All Current Teams" width="20"/> <span class="align-middle">All Current NBA Teams</span> </a>,
 <a class="sidebar-link py-1 mx-1" href="https://www.2kratings.com/teams/atlanta-hawks"> <img alt="Atlanta Hawks" class="lazy p-1 pl-0" data-src="https://www.2kratings.com/wp-content/uploads/Atlanta-Hawks-Secondary-Logo.svg" height="24" src="/wp-content/uploads/1x1.png" title="Atlanta Hawks" width="24"/> Atlanta Hawks </a>,
 <a class="sidebar-link py-1 mx-1" href="https://www.2kratings.com/teams/boston-celtics"> <img alt="Boston Celtics" class="lazy p-1 pl-0" data-src="https://www.2kratings.com/wp-content/uploads/Boston-Celtics-Secondary-Logo.svg" height="24" src="/wp-content/uploads/1x1.png" title="Boston Celtics" width="24"/> Boston Celtics </a>]

In [ ]:
team_links_list = []
rating_player_list = []

for team in team_links:
    if (re.search('/teams/',team['href'])is not None):
        team_links_list.append(team['href'])
        team_name = team.find('img')['alt']
        print(team_name)

        # url_team = team_links_list[0]
        url_team = team['href']

        # Make a GET request to the website
        response_team = requests.get(
            url_team,
            headers={'User-Agent': 'Mozilla/5.0'}
            )
        soup_team = BeautifulSoup(response_team.content, "lxml")

        # find all teaam players
        players_spans = soup_team.find_all('span', {"class": "entry-font"})



        for player_span in players_spans:
            if (re.search('teams',player_span.find('a')['href']) is None):
                
                # I wait 1 second per player so I don't have problems with requests
                time.sleep(1)
                # Make a GET request to the website
                url_player = player_span.find('a')['href']
                response_player = requests.get(
                    url_player,
                    headers={'User-Agent': 'Mozilla/5.0'}
                    )
                soup_player = BeautifulSoup(response_player.content, "lxml")
                
                # list of all the players
                rating_player = {}
                rating_player['Team Name'] = team_name
                rating_player['Player Name'] = player_span.text

                for block in soup_player.find_all('ul',{'class':'list-group list-no-bullet'}):
                    for item in block.find_all('li'):
                        skill = ' '.join(item.text.split()[1:])
                        rating_value = item.text.split()[0]
                        rating_player[skill] = rating_value
                rating_player_list.append(rating_player)
                        
        

In [ ]:
# team_links
# players_spans
df = pd.DataFrame(rating_player_list)
df.to_csv('ratings_NBA2k24.csv')